## Prov-GigaPath Demo

This notebook provides a quick walkthrough of the Prov-GigaPath models. We will start by demonstrating how to download the Prov-GigaPath models from HuggingFace. Next, we will show an example of pre-processing a slide. Finally, we will demonstrate how to run Prov-GigaPath on the sample slide.

### Prepare HF Token

To begin, please request access to the model from our HuggingFace repository: https://huggingface.co/prov-gigapath/prov-gigapath.

Once approved, set the HF_TOKEN to access the model.

In [29]:
import os

# Please set your Hugging Face API token
# os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"

homedir_path = os.path.expanduser("~")
assert ("HF_TOKEN" in os.environ) or os.path.exists(f"{homedir_path}/.cache/huggingface/token"), "Please set the HF_TOKEN environment variable to your Hugging Face API token or make sure the token is cached in ~/.cache/huggingface/token"

In [30]:
PROJECT_DIR = ".."

local_dir_name = "sample_data"
local_dir = os.path.join(PROJECT_DIR, local_dir_name)

slide_file_name = "PROV-000-000001.ndpi"
slide_hf_path = os.path.join(local_dir_name, slide_file_name)
slide_path = os.path.join(local_dir, "PROV-000-000001.ndpi")


tile_save_dir = os.path.join(local_dir, "outputs/preprocessing")
specific_slide_tiles_dir = f"{tile_save_dir}/output/{slide_file_name}"
os.makedirs(specific_slide_tiles_dir, exist_ok=True)

### Download a sample slide

In [32]:
import huggingface_hub

huggingface_hub.hf_hub_download(
    "prov-gigapath/prov-gigapath",
    filename=slide_hf_path,
    local_dir=PROJECT_DIR,
    force_download=True,
)

PROV-000-000001.ndpi: 100%|██████████| 424M/424M [00:30<00:00, 14.0MB/s] 


'../sample_data/PROV-000-000001.ndpi'

### Tiling

Whole-slide images are giga-pixel in size. To efficiently process these enormous images, we use a tiling technique that divides them into smaller, more manageable tile images. As an example, we demonstrate how to process a single slide.

NOTE: Prov-GigaPath is trained with slides preprocessed at 0.5 MPP. Ensure that you use the appropriate level for the 0.5 MPP.

In [33]:
from gigapath.pipeline import tile_one_slide

print("NOTE: Prov-GigaPath is trained with 0.5 mpp preprocessed slides. Please make sure to use the appropriate level for the 0.5 MPP")
tile_one_slide(slide_path, save_dir=tile_save_dir, level=1)

NOTE: Prov-GigaPath is trained with 0.5 mpp preprocessed slides. Please make sure to use the appropriate level for the 0.5 MPP
Processing slide ../sample_data/PROV-000-000001.ndpi at level 1 with tile size 256. Saving to ../sample_data/outputs/preprocessing.
('slide_id', 'tile_id', 'image', 'label', 'tile_x', 'tile_y', 'occupancy')
Slide ../sample_data/PROV-000-000001.ndpi has been tiled. 1068 tiles saved to ../sample_data/outputs/preprocessing/output/PROV-000-000001.ndpi.


### Load the tile images

In [34]:
import os

# load image tiles
image_paths = [
    os.path.join(specific_slide_tiles_dir, img)
    for img in os.listdir(specific_slide_tiles_dir)
    if img.endswith(".png")
]

print(f"Found {len(image_paths)} image tiles")

Found 1068 image tiles


### Load the Prov-GigaPath model (tile and slide encoder models)

In [5]:
from gigapath.pipeline import load_tile_slide_encoder

# Load the tile and slide encoder models
# NOTE: The CLS token is not trained during the slide-level pretraining.
# Here, we enable the use of global pooling for the output embeddings.
tile_encoder, slide_encoder_model = load_tile_slide_encoder(global_pool=True)

Tile encoder param # 1134953984
dilated_ratio:  [1, 2, 4, 8, 16]
segment_length:  [1024, 5792, 32768, 185363, 1048576]
Number of trainable LongNet parameters:  85148160
Global Pooling: True


slide_encoder.pth: 100%|██████████| 345M/345M [00:01<00:00, 235MB/s] 


 Successfully Loaded Pretrained GigaPath model from hf_hub:prov-gigapath/prov-gigapath 
Slide encoder param # 86330880


### Run tile-level inference

In [6]:
from gigapath.pipeline import run_inference_with_tile_encoder

tile_encoder_outputs = run_inference_with_tile_encoder(image_paths, tile_encoder)

for k in tile_encoder_outputs.keys():
    print(f"tile_encoder_outputs[{k}].shape: {tile_encoder_outputs[k].shape}")

Running inference with tile encoder: 100%|██████████| 9/9 [00:09<00:00,  1.10s/it]

tile_encoder_outputs[tile_embeds].shape: torch.Size([1068, 1536])
tile_encoder_outputs[coords].shape: torch.Size([1068, 2])


### Run slide-level inference

In [7]:
from gigapath.pipeline import run_inference_with_slide_encoder
# run inference with the slide encoder
slide_embeds = run_inference_with_slide_encoder(slide_encoder_model=slide_encoder_model, **tile_encoder_outputs)
print(slide_embeds.keys())

dict_keys(['layer_0_embed', 'layer_1_embed', 'layer_2_embed', 'layer_3_embed', 'layer_4_embed', 'layer_5_embed', 'layer_6_embed', 'layer_7_embed', 'layer_8_embed', 'layer_9_embed', 'layer_10_embed', 'layer_11_embed', 'layer_12_embed', 'last_layer_embed'])
